In [1]:
import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn 
import torch.optim as optim

In [2]:
HIDDEN_SIZE = 128       # number of neurons
BATCH_SIZE =16          # the count of episodes per iteration
PERCENTILE = 70         # the percentile of episodes' total rewards that we use for elite episode filtering
                        ## we will leave the top 30% of episodes sorted by reward

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        # one hidden layer NN, with recified linear unit (ReLU) and 128 nuerons
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    # env.reset() returns a tuple of (observation, info) instead of observatrion
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1)
    
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _, _ = env.step(action)
        # add dimension to next_obs
        # next_obs = [next_obs]
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
env = gym.make('CartPole-v0')

obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

c:\Users\woojin\anaconda3\envs\network\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [8]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

In [9]:
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss%.3f, reward_mean=%.1f, rw_bound=%.1f"
          %(iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break

writer.close()
    

C:\Users\woojin\AppData\Local\Temp\ipykernel_13696\3255438199.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  obs_v = torch.FloatTensor([obs])


0: loss0.695, reward_mean=16.9, rw_bound=19.0
1: loss0.684, reward_mean=23.4, rw_bound=23.5
2: loss0.667, reward_mean=25.6, rw_bound=23.5
3: loss0.647, reward_mean=35.5, rw_bound=38.5
4: loss0.633, reward_mean=32.6, rw_bound=28.0
5: loss0.623, reward_mean=29.6, rw_bound=34.0
6: loss0.613, reward_mean=39.7, rw_bound=48.5
7: loss0.624, reward_mean=37.1, rw_bound=37.5
8: loss0.584, reward_mean=48.8, rw_bound=64.5
9: loss0.589, reward_mean=61.1, rw_bound=71.0
10: loss0.582, reward_mean=70.6, rw_bound=84.5
11: loss0.576, reward_mean=57.3, rw_bound=62.5
12: loss0.570, reward_mean=54.8, rw_bound=59.5
13: loss0.557, reward_mean=57.3, rw_bound=63.0
14: loss0.571, reward_mean=46.5, rw_bound=51.5
15: loss0.562, reward_mean=59.9, rw_bound=67.0
16: loss0.538, reward_mean=56.8, rw_bound=61.0
17: loss0.536, reward_mean=59.9, rw_bound=61.0
18: loss0.536, reward_mean=75.1, rw_bound=95.0
19: loss0.531, reward_mean=68.5, rw_bound=85.0
20: loss0.512, reward_mean=66.6, rw_bound=76.5
21: loss0.525, reward_m